In [1]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat
# from langchain.llms.gigachat import GigaChat
from dotenv import load_dotenv
import json
import re
import os 
load_dotenv()
GIGACHAT_CLIENT_SECRET = os.getenv("GIGACHAT_CLIENT_SECRET_B64")

In [2]:
from docx import Document
from docx.shared import Inches

In [4]:
from docx import Document
# from docx.shared import Pt
# from docx.enum.text import WD_ALIGN_PARAGRAPH

# Open an existing document
doc = Document('iskovoe-zayavlenie-o-rastorzhenii-braka-i-razd.docx')

In [ ]:
def find_blank_spots(text):
    pattern = r'_+'  # Шаблон для поиска одного или более подчеркиваний
    matches = re.finditer(pattern, text)
    
    results = []
    for match in matches:
        start = match.start()
        end = match.end()
        results.append((start, end))
        
    return results

In [27]:
# # Пример использования:
# doc_text = """
# Заявитель:_______
# Дата:___________
# """
# # for line in lines:
blanks = find_blank_spots(doc_text)
print(blanks)
if blanks:
    for start, end in blanks:
        print(f"Найден пропуск с позиции {start} до {end-1} в строке: '{doc_text[start:end-1]}'")

print(doc_text[11:17])

[(11, 18), (24, 35)]
Найден пропуск с позиции 11 до 17 в строке: '______'
Найден пропуск с позиции 24 до 34 в строке: '__________'
______


### Использование языковой модели для заполнения пропусков в тексте

In [31]:
chat = GigaChat(credentials=GIGACHAT_CLIENT_SECRET, verify_ssl_certs=False) #model='GigaChat-Pro'

In [128]:
extraction_prompt = """Ты помогаешь заполнить юридический документ.
Нужно переписать строку, заменив ___ на шалон, описывающий пропущенные данные.
Шаблон должен быть понятен пользователю, который будет заполнять документ.
Например, [[Дата регистрации]]
Инструкция:
1. Сначала определи, какие данные нужно вставить вместо ___
2. Перепиши строку используя [[Шаблон]]
3. Верни исходную строку полностью, вставив в нее шаблон


Пример 1:
Строка с пропуском:
Представитель истца: ________________________________
(Ф.И.О., адрес)

Ответ:
Представитель истца: [[ФИО и адрес истца]]

Пример 2:
Строка с пропуском:
Мировому судье ___________________________
Областного района ________________________

Ответ:
Мировому судье [[ФИО мирового судьи]]
Областного района [[Номер областного района, в котором состоится суд]]

Пример 3:
Строка с пропуском:
                                 судебного участка N ______________________

                                 Истец: ___________________________________
                                                  (Ф.И.О., адрес)

Ответ:
                                 судебного участка N [[Номер судебного участка]]

                                 Истец: [[ФИО и адрес истца]]

Пример 4:
Строка с пропуском:
Стороны определили, что ответчик вправе проводить с 
несовершеннолетним _________ (фамилия, имя, отчество ребенка) половину

Ответ:
Стороны определили, что ответчик вправе проводить с несовершеннолетним [[ФИО ребенка]] половину

Пример 5:
Строка с пропуском:
В период брака у нас возникли следующие обязательства: ________________

Ответ:
В период брака у нас возникли следующие обязательства: [[Возникшие в браке обязательства]]
"""

In [79]:
# messages = [
#     SystemMessage(
#         content=extraction_prompt
#     )
# ]

In [73]:
text = doc.paragraphs[9:13]
batch = []
for elem in text:
    batch.append(elem.text)

batch

['                                 Третье лицо: орган опеки и попечительства,',
 '                                 место нахождения: ________________________',
 '                                 __________________________________________',
 '']

In [74]:
input_text = '\n'.join(batch)
print(input_text)

                                 Третье лицо: орган опеки и попечительства,
                                 место нахождения: ________________________
                                 __________________________________________



In [75]:
messages.append(HumanMessage(content=input_text))
res = chat(messages)
messages.append(res)
print(res.content)

                                Третье лицо: орган опеки и попечительства,
                                 место нахождения: [[Адрес органа опеки и попечительства]]


#### Конвейер для проверки работоспособности на всем документе

In [81]:
final_text = []
paragraphs = [p.text for p in doc.paragraphs]
# paragraphs

In [85]:
batch_size = 4
length = len(paragraphs)
for i in range(0, length, batch_size):
    batch = paragraphs[i:min(i + batch_size, length)]
    input_text = '\n'.join(batch)
    messages = [SystemMessage(content=extraction_prompt)]
    messages.append(HumanMessage(content=input_text))
    res = chat(messages)
    final_text.append(res.content)
    # messages.append(res) # Сохранение истории запросов
    print(f'Запрос \n{input_text}')
    print(f'Ответ\n\n{res.content}')
    



Запрос 
Мировому судье Иванову Андрею Васильевичу
                                 судебного участка N ______________________

                                 Истец: ___________________________________
Ответ

Мировому судье Иванову Андрею Васильевичу
                                 судебного участка N [[Номер судебного участка]]

                                 Истец: [[ФИО и адрес истца]]
Запрос 
                                                  (Ф.И.О., адрес)

Мировому судье Иванову Андрею Васильевичу
                                                  (Ф.И.О., адрес)
Ответ

Мировому судье Иванову Андрею Васильевичу
                                                  (Ф.И.О., адрес)
Запрос 

                                 Третье лицо: орган опеки и попечительства,
                                 место нахождения: ________________________
                                 __________________________________________
Ответ

Третье лицо: орган опеки и попечительства,
место нахождения: [

In [102]:
for i in range(len(final_text)):
    batch = final_text[i]
    if '_' in batch:   
        messages = [SystemMessage(content=extraction_prompt)]
        messages.append(HumanMessage(content=batch))
        res = chat(messages)
        final_text[i] = res.content
        # messages.append(res) # Сохранение истории запросов
        print(f'Запрос \n{batch}')
        print(f'Ответ\n\n{res.content}')


Запрос 
Соглашение о добровольном разделе имущества, являющегося общей совместной собственностью, между нами не достигнуто. В период брака нами совместно приобретено следующее имущество: ________________________________
Ответ

Соглашение о добровольном разделе имущества, являющегося общей совместной собственностью, между нами не достигнуто. В период брака нами совместно приобретено следующее имущество: ________________________________
Запрос 
____________________________________________________________________,
общая стоимость которого составляет _________________________ руб.
    В период брака у нас возникли следующие обязательства: ________________
Ответ

____________________________________________________________________,
общая стоимость которого составляет _________________________ руб.
Запрос 
__________________________________________________________________________.
    Перед  расторжением  брака  ответчица  без моего согласия (вопреки моей
воле) произвела отчуждение общего им

In [96]:
print('\n\n'.join(final_text))

Мировому судье Иванову Андрею Васильевичу
                                 судебного участка N [[Номер судебного участка]]

                                 Истец: [[ФИО и адрес истца]]

Мировому судье Иванову Андрею Васильевичу
                                                  (Ф.И.О., адрес)

Третье лицо: орган опеки и попечительства,
место нахождения: [[Адрес органа опеки и попечительства]]

Цена иска: [[Сумма иска]]

С ответчицей(ком) я вступил(а) в брак [[Число, месяц, год заключения брака]].

От брака имеем ребенка (детей) [[Имя, дата рождения ребенка]]

Совместная жизнь с ответчицей(ком) не сложилась [[Указать причины разрыва отношений]]

Брачные отношения между нами прекращены с [[Год и месяц развода]], общее

хозяйство не ведется.
Примирение между мной и ответчицей(ком) невозможно. По вопросу о содержании и воспитании ребенка (детей) спора нет [[ФИО истца]]

[[Описание документа или ситуации]]

Стороны определили, что ответчик вправе проводить с несовершеннолетним _________ (ф

In [105]:
new_doc = Document()

# Добавляем абзац с текстом в документ\
for text_block in final_text:
    print(text_block)
    new_doc.add_paragraph(text_block)

# Сохраняем документ
new_doc.save('example.docx')

Мировому судье Иванову Андрею Васильевичу
                                 судебного участка N [[Номер судебного участка]]

                                 Истец: [[ФИО и адрес истца]]
Мировому судье Иванову Андрею Васильевичу
                                                  (Ф.И.О., адрес)
Третье лицо: орган опеки и попечительства,
место нахождения: [[Адрес органа опеки и попечительства]]
Цена иска: [[Сумма иска]]
С ответчицей(ком) я вступил(а) в брак [[Число, месяц, год заключения брака]].
От брака имеем ребенка (детей) [[Имя, дата рождения ребенка]]
Совместная жизнь с ответчицей(ком) не сложилась [[Указать причины разрыва отношений]]
Брачные отношения между нами прекращены с [[Год и месяц развода]], общее
хозяйство не ведется.
Примирение между мной и ответчицей(ком) невозможно. По вопросу о содержании и воспитании ребенка (детей) спора нет [[ФИО истца]]
[[Описание документа или ситуации]]
Стороны определили, что ответчик вправе проводить с несовершеннолетним [ФИО ребенка] половину

### Извлечение сущностей издокумента и генерация вопросов

In [131]:
doc_with_entities = Document('processed_arenda.docx')

In [134]:
text_with_entities = [p.text for p in doc_with_entities.paragraphs]
text_with_entities = '\n'.join(text_with_entities)
print(text_with_entities)

ДОГОВОР НАЙМА КВАРТИРЫ

 Гр. [[ФИО наймодателя]], паспорт: серия [[Серия паспорта наймодателя]], № [[Номер паспорта наймодателя]], выданный [[Место выдачи паспорта наймодателя]], проживающий по адресу: [[Адрес наймодателя]], именуемый в дальнейшем «Наймодатель», с одной стороны, и гр. [[ФИО нанимателя]], паспорт: серия [[Серия паспорта нанимателя]], № [[Номер паспорта нанимателя]], выданный [[Место выдачи паспорта нанимателя]], проживающий по адресу: [[Адрес нанимателя]], именуемый в дальнейшем «Наниматель», с другой стороны, именуемые в дальнейшем «Стороны», заключили настоящий договор, в дальнейшем «Договор», о нижеследующем:
1. Наймодатель предоставляет Нанимателю во временное владение и пользование за плату принадлежащую ему по праву собственности квартиру, кадастровый номер: [[Кадастровый номер квартиры]], находящуюся по адресу: [[Адрес квартиры]], общая площадь [[Площадь квартиры]] кв.м, состоящую из [[Количество комнат квартиры]]. Квартира предоставляется Нанимателю для проживан

In [135]:
pattern = r'\[\[(.*?)\]\]'

# Поиск всех подстрок
matches = re.findall(pattern, text_with_entities)

# Вывод результатов
print(len(matches))
print(len(set(matches)))
print(matches)

20
20
['ФИО наймодателя', 'Серия паспорта наймодателя', 'Номер паспорта наймодателя', 'Место выдачи паспорта наймодателя', 'Адрес наймодателя', 'ФИО нанимателя', 'Серия паспорта нанимателя', 'Номер паспорта нанимателя', 'Место выдачи паспорта нанимателя', 'Адрес нанимателя', 'Кадастровый номер квартиры', 'Адрес квартиры', 'Площадь квартиры', 'Количество комнат квартиры', 'Срок найма в месяцах', 'Размер оплаты в месяц', 'Общая стоимость договора', 'Способ оплаты и срок внесения платы', 'Дата и время оплаты', 'Количество экземпляров Договора']


In [136]:
question_prompt = """
Тебе нужно помочь пользователю заполнить юридические документы. 
У тебя есть название поля из документа. Используя его, сгенерируй вопрос пользователю,
чтобы уточнить его данные. Приведи пример заполнения, чтобы пользователю было проще понять 
нужный формат.
Будь вежлив.

Пример 1:
Поле:
Сумма иска

Вопрос пользователю:
Введите, пожалуйста, сумму иска в рублях. Например: 12500 руб

Пример 2:
Поле:
ФИО и адрес истца

Вопрос пользователю:
Введите, пожалуйста, ФИО истца в именительном падеже и адрес его регистрации. 
Например: Васнецов Виталий Сергеевич г. Санкт-Петербург ул.Дыбенко д. 2 кв. 12
"""

In [34]:
messages = [SystemMessage(content=question_prompt)]
messages.append(HumanMessage(content=f"Поле: Номер судебного участка\n Вопрос пользователю:"))
res = chat(messages)
res.content

'Введите, пожалуйста, номер судебного участка, где будет рассматриваться дело. Например: 34'

In [137]:
fill_dict = {}
for entity in matches:
    messages = [SystemMessage(content=question_prompt)]
    messages.append(HumanMessage(content=f"Поле: {entity}\n Вопрос пользователю:"))
    res = chat(messages)
    print(f'Entity:\n{entity}\n\n')
    print(f'Question:\n{res.content}')
    fill_dict.setdefault(entity, {})['question'] = res.content

AUTHENTICATION ERROR


Entity:
ФИО наймодателя


Question:
Введите, пожалуйста, фамилию, имя и отчество (при наличии) лица, предоставляющего жилье в аренду. Пример: Иванов Иван Иванович
Entity:
Серия паспорта наймодателя


Question:
Укажите, пожалуйста, серию паспорта наймодателя. Например: 4607
Entity:
Номер паспорта наймодателя


Question:
Введите, пожалуйста, номер паспорта наймодателя. Например: 123456
Entity:
Место выдачи паспорта наймодателя


Question:
Укажите, пожалуйста, место выдачи паспорта наймодателя. Например: Управление по вопросам миграции ГУ МВД России по городу Москве
Entity:
Адрес наймодателя


Question:
Введите, пожалуйста, полный адрес наймодателя, включая город, улицу, номер дома и квартиры.
Entity:
ФИО нанимателя


Question:
Введите, пожалуйста, фамилию, имя и отчество (при наличии) лица, которое будет проживать в квартире или доме. Например: Иванов Иван Иванович
Entity:
Серия паспорта нанимателя


Question:
Укажите, пожалуйста, серию паспорта нанимателя. Например: 46 03
Entity:
Номер 

In [139]:
with open('schema.json', 'w', encoding='utf-8') as json_file:
    json.dump(fill_dict, json_file, indent=4)

### Заполнение документа с использованием информации пользователя

#### Вариант заполнения с регулярными выражениями
Не совсем точен, если пользователь ошибся при введении данных. 
Можно дополнить валидацией введенных пользователем полей.

In [46]:
with open("schema.json", "r", encoding='utf-8') as f:
    schema = json.load(f)

In [49]:
str_document = Document('example.docx')
str_document = '\n'.join(p.text for p in str_document.paragraphs)

In [56]:
def replace_substrings(input_string, schema):
    # Паттерн для поиска подстрок вида [[...]]
    pattern = r'\[\[(.*?)\]\]'
    
    # Функция замены подстрок
    def replace_value(match):
        key = match.group(1)  # Содержимое внутри [[...]]
        return schema[key].get('user_answer', f'[[{key}]]') #replacement_dict[key]  # Возвращаем значение из словаря или оставляем как есть
    
    # Заменяем подстроки с помощью регулярных выражений и функции замены
    result_string = re.sub(pattern, replace_value, input_string)
    
    return result_string


In [57]:
output_string = replace_substrings(str_document, schema)

In [58]:
print(output_string)

Мировому судье Иванову Андрею Васильевичу
                                 судебного участка N 142

                                 Истец: Курьянов Валерий Андреевич г. Москва ул. Космонавтов д.23 кв. 15
Мировому судье Иванову Андрею Васильевичу
                                                  (Ф.И.О., адрес)
Третье лицо: орган опеки и попечительства,
место нахождения: г. Нептун ул. Красная д.23
Цена иска: 12300 руб.
С ответчицей(ком) я вступил(а) в брак 14.03.2023.
От брака имеем ребенка (детей)  Андрей 01.01.2020
Совместная жизнь с ответчицей(ком) не сложилась Неошлись взглядами на политическое устройство Марса, не смогли построить дом в Кипре
Брачные отношения между нами прекращены с январь 2022, общее
хозяйство не ведется.
Примирение между мной и ответчицей(ком) невозможно. По вопросу о содержании и воспитании ребенка (детей) спора нет Андреев Андрей Андреевич
?
Стороны определили, что ответчик вправе проводить с несовершеннолетним [ФИО ребенка] половину всего времени, свободного

#### Вариант заполнения с LLM

In [64]:
filling_prompt = """Ты - профессиональный юрист с большим опытом.
Ты знаешь все тонсти заполнения юридических документов.
Тебе будет передан блок текста с пропуском и информация для его заполнения.
Нужно переписать заполнить все пропуски в строке.

Пример 1
Строка документа:
Истец: [[ФИО истца]]
Брак был расторгнут по обоюдному согласию: [[Дата расторжения брака]]
Данные пользователя:
ФИО истца: Андреев Михаил Валерьевич
Дата расторжения брака: 22.02.2019

Ответ:
Истец: Андреев Михаил Валерьевич
Брак был расторгнут по обоюдному согласию: 22.02.2019
"""

In [60]:
document = Document('example.docx')
paragraphs = [p.text for p in document.paragraphs]

In [65]:
batch = '\n'.join(paragraphs[0:4])

user_data = []

pattern = r'\[\[(.*?)\]\]'
matches = re.findall(pattern, batch)
for elem in matches:
    
    data = schema[elem].get('user_answer', 'Данные неизвестны')
    hint = f'{elem}: {data}'
    user_data.append(hint)

user_data = '\n'.join(user_data)
print(user_data)

Номер судебного участка: 142
ФИО и адрес истца: Курьянов Валерий Андреевич г. Москва ул. Космонавтов д.23 кв. 15
Адрес органа опеки и попечительства: г. Нептун ул. Красная д.23
Сумма иска: 12300 руб.


In [66]:
input_text = f'Строка документа:\n{batch}\n\n Данные пользователя:{user_data}\n\n Ответ:'
messages = [SystemMessage(content=filling_prompt)]
messages.append(HumanMessage(content=input_text))
res = chat(messages)
print(res.content)

AUTHENTICATION ERROR


In [68]:
print(batch)
print('======================')
print(res.content)

Мировому судье Иванову Андрею Васильевичу
                                 судебного участка N [[Номер судебного участка]]

                                 Истец: [[ФИО и адрес истца]]
Мировому судье Иванову Андрею Васильевичу
                                                  (Ф.И.О., адрес)
Третье лицо: орган опеки и попечительства,
место нахождения: [[Адрес органа опеки и попечительства]]
Цена иска: [[Сумма иска]]
Мировому судье Иванову Андрею Васильевичу
                                 судебного участка N 142

                                 Истец: Курьянов Валерий Андреевич г. Москва ул. Космонавтов д.23 кв. 15
Мировому судье Иванову Андрею Васильевичу
                                                  (Ф.И.О., адрес)
Третье лицо: орган опеки и попечительства,
место нахождения: г. Нептун ул. Красная д.23
Цена иска: 12300 руб.


In [69]:
filled_text = []

batch_size = 4
length = len(paragraphs)
for i in range(0, length, batch_size):
    # batch = paragraphs[i:min(i + batch_size, length)]
    batch = '\n'.join(paragraphs[i:min(i + batch_size, length)])

    user_data = []

    pattern = r'\[\[(.*?)\]\]'
    matches = re.findall(pattern, batch)
    for elem in matches:
        
        data = schema[elem].get('user_answer', 'Данные неизвестны')
        hint = f'{elem}: {data}'
        user_data.append(hint)

    input_text = f'Строка документа:\n{batch}\n\n Данные пользователя:{user_data}\n\n Ответ:'
    messages = [SystemMessage(content=filling_prompt)]
    messages.append(HumanMessage(content=input_text))
    res = chat(messages)
    filled_text.append(res.content)
    # messages.append(res) # Сохранение истории запросов
    print(f'Запрос \n{input_text}')
    print(f'Ответ\n\n{res.content}')

Запрос 
Строка документа:
Мировому судье Иванову Андрею Васильевичу
                                 судебного участка N [[Номер судебного участка]]

                                 Истец: [[ФИО и адрес истца]]
Мировому судье Иванову Андрею Васильевичу
                                                  (Ф.И.О., адрес)
Третье лицо: орган опеки и попечительства,
место нахождения: [[Адрес органа опеки и попечительства]]
Цена иска: [[Сумма иска]]

 Данные пользователя:['Номер судебного участка: 142', 'ФИО и адрес истца: Курьянов Валерий Андреевич г. Москва ул. Космонавтов д.23 кв. 15', 'Адрес органа опеки и попечительства: г. Нептун ул. Красная д.23', 'Сумма иска: 12300 руб.']

 Ответ:
Ответ

Мировому судье Иванову Андрею Васильевичу
                                 судебного участка N 142

                                 Истец: Курьянов Валерий Андреевич г. Москва ул. Космонавтов д.23 кв. 15
Мировому судье Иванову Андрею Васильевичу
                                                  (Ф.И.

In [70]:
final_document = Document()

# Добавляем абзац с текстом в документ\
for text_block in filled_text:
    print(text_block)
    final_document.add_paragraph(text_block)

# Сохраняем документ
final_document.save('final_document.docx')

Мировому судье Иванову Андрею Васильевичу
                                 судебного участка N 142

                                 Истец: Курьянов Валерий Андреевич г. Москва ул. Космонавтов д.23 кв. 15
Мировому судье Иванову Андрею Васильевичу
                                                  (Ф.И.О., адрес)
Третье лицо: орган опеки и попечительства,
место нахождения: г. Нептун ул. Красная д.23
Цена иска: 12300 руб.
С ответчицей(ком) я вступил(а) в брак 14.03.2023.
От брака имеем ребенка (детей) Андрей 01.01.2020.
Совместная жизнь с ответчицей(ком) не сложилась из-за неошлись взглядами на политическое устройство Марса, не смогли построить дом в Кипре.
Брачные отношения между нами прекращены с января 2022 года, общее
хозяйство не ведется.
Примирение между мной и ответчицей(ком) невозможно. По вопросу о содержании и воспитании ребенка (детей) спора нет Андреев Андрей Андреевич
[[Описание документа или ситуации]]
Стороны определили, что ответчик вправе проводить с несовершеннолетним [Ф

### Черновик с тестами

In [107]:
import re
def is_valid_string(input_string):
    pattern = r'^[_. ]+$'
    return not bool(re.match(pattern, input_string))

def shorten_blanks(input_string):
    # Паттерн для поиска длинных пропусков
    
    if is_valid_string(input_string):
        pattern = r'(_{2,})'
        # Заменяем длинные пропуски на сокращенный вариант
        output_string = re.sub(pattern, '___', input_string)
        
        return output_string
    else:
        return ''

In [110]:
doc = Document('iskovoe-zayavlenie-o-rastorzhenii-braka-i-razd.docx')

final_text = []
paragraphs = [p.text for p in doc.paragraphs]
paragraphs_upd = [shorten_blanks(p.text) for p in doc.paragraphs]

In [111]:
print('\n'.join(paragraphs_upd))

Мировому судье Иванову Андрею Васильевичу
                                 судебного участка N ___

                                 Истец: ___
                                                  (Ф.И.О., адрес)

Мировому судье Иванову Андрею Васильевичу
                                                  (Ф.И.О., адрес)

                                 Третье лицо: орган опеки и попечительства,
                                 место нахождения: ___


                                 Цена иска: ___

                             ИСКОВОЕ ЗАЯВЛЕНИЕ
                  о расторжении брака и разделе имущества

    С ответчицей(ком) я вступил(а) в брак ___.
                                            (число, месяц, год)

    От брака имеем ребенка (детей) ___

             (имя, число, месяц, год рождения ребенка (детей))

    Совместная жизнь с ответчицей(ком) не сложилась ___

                             (указать причины)

    Брачные отношения между нами прекращены с ___, общее
              

In [83]:
print('\n'.join(paragraphs))

Мировому судье Иванову Андрею Васильевичу
                                 судебного участка N ______________________

                                 Истец: ___________________________________
                                                  (Ф.И.О., адрес)

Мировому судье Иванову Андрею Васильевичу
                                                  (Ф.И.О., адрес)

                                 Третье лицо: орган опеки и попечительства,
                                 место нахождения: ________________________
                                 __________________________________________

                                 Цена иска: _______________________________

                             ИСКОВОЕ ЗАЯВЛЕНИЕ
                  о расторжении брака и разделе имущества

    С ответчицей(ком) я вступил(а) в брак _______________________.
                                            (число, месяц, год)

    От брака имеем ребенка (детей) ________________________________________
_________

In [125]:
import re
def is_valid_string(input_string):
    pattern = r'^[_. ]+$'
    return not bool(re.match(pattern, input_string))

def shorten_blanks(input_string):
    # Паттерн для поиска длинных пропусков
    
    if is_valid_string(input_string):
        pattern = r'(_{2,})'
        # Заменяем длинные пропуски на сокращенный вариант
        output_string = re.sub(pattern, '___', input_string)
        
        return output_string
    else:
        return ''

In [124]:
is_valid_string('asa')

True

In [130]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat
# from langchain.llms.gigachat import GigaChat
# from prompts import extraction_prompt
# from utils import shorten_blanks
from docx import Document
from dotenv import load_dotenv
import os 
load_dotenv()

GIGACHAT_CLIENT_SECRET = os.getenv("GIGACHAT_CLIENT_SECRET_B64")
BATCH_SIZE = 4

chat = GigaChat(credentials=GIGACHAT_CLIENT_SECRET, verify_ssl_certs=False) #model='GigaChat-Pro'
doc = Document('dogovor_arenda.docx')

final_text = []
paragraphs = [shorten_blanks(p.text) for p in doc.paragraphs]


length = len(paragraphs)
for i in range(0, length, BATCH_SIZE):
    batch = paragraphs[i:min(i + BATCH_SIZE, length)]
    input_text = '\n'.join(batch)
    input_text = f'Строка с пропуском: {input_text}\nОтвет:'
    messages = [SystemMessage(content=extraction_prompt)]
    messages.append(HumanMessage(content=input_text))
    res = chat(messages)
    final_text.append(res.content)
    # messages.append(res) # Сохранение истории запросов
    print(f'Запрос \n{input_text}')
    print(f'Ответ\n\n{res.content}')


# for i in range(len(final_text)):
#     batch = final_text[i]
#     if '_' in batch:   
#         messages = [SystemMessage(content=extraction_prompt)]
#         messages.append(HumanMessage(content=batch))
#         res = chat(messages)
#         final_text[i] = res.content
#         # messages.append(res) # Сохранение истории запросов
#         print(f'Запрос \n{batch}')
#         print(f'Ответ\n\n{res.content}')

new_doc = Document()

# Добавляем абзац с текстом в документ\
for text_block in final_text:
    print(text_block)
    new_doc.add_paragraph(text_block)

# Сохраняем документ
new_doc.save('processed_arenda.docx')

Запрос 
Строка с пропуском: ДОГОВОР НАЙМА КВАРТИРЫ


 Гр. ___, паспорт: серия ___, № ___, выданный ___, проживающий по адресу: ___, именуемый в дальнейшем «Наймодатель», с одной стороны, и гр. ___, паспорт: серия ___, № ___, выданный ___, проживающий по адресу: ___, именуемый в дальнейшем «Наниматель», с другой стороны, именуемые в дальнейшем «Стороны», заключили настоящий договор, в дальнейшем «Договор», о нижеследующем: 
Ответ:
Ответ

ДОГОВОР НАЙМА КВАРТИРЫ

 Гр. [[ФИО наймодателя]], паспорт: серия [[Серия паспорта наймодателя]], № [[Номер паспорта наймодателя]], выданный [[Место выдачи паспорта наймодателя]], проживающий по адресу: [[Адрес наймодателя]], именуемый в дальнейшем «Наймодатель», с одной стороны, и гр. [[ФИО нанимателя]], паспорт: серия [[Серия паспорта нанимателя]], № [[Номер паспорта нанимателя]], выданный [[Место выдачи паспорта нанимателя]], проживающий по адресу: [[Адрес нанимателя]], именуемый в дальнейшем «Наниматель», с другой стороны, именуемые в дальнейшем «Сто

Запрос 
Строка с пропуском: 1. Наймодатель предоставляет Нанимателю во временное владение и пользование за плату принадлежащую ему по праву собственности квартиру, кадастровый номер: ___ , находящуюся по адресу: ___ , общая площадь ___ кв.м, состоящую из ___ комнат (далее Квартира). Квартира предоставляется Нанимателю для проживания.
2. Срок найма ___ . 
3. Оплата найма Квартиры производится ежемесячно в сумме ___ рублей. Общая сумма Договора составляет ___ рублей. Наймодатель обязан вносить плату на счет Наймодателя ___ или передавать ее лично под расписку не позднее ___ числа каждого месяца.
3.Наймодатель не может использовать Квартиру в иных целях, чем указано в п. 1 Договора, а также не может сдавать квартиру в поднаем. В Квартире имеют право проживать члены семьи Нанимателя.
Ответ:
Ответ

1. Наймодатель предоставляет Нанимателю во временное владение и пользование за плату принадлежащую ему по праву собственности квартиру, кадастровый номер: [[Кадастровый номер квартиры]], находящу